## Automatic Differentiation with <kbd>TORCH.AUTOGRAD</kbd>

当我们要训练一个neural network时，我们频繁用到**back propagation**。这个算法会根据给定parameter的loss函数的**gradient**调整parameters(model weight)。

Pytorch用内建的叫做<kbd>TORCH.AUTOGRAD</kbd>的differentiation engine来计算这些parameter，它支持对任何的计算图（computational graph）进行自动计算gradient。

考虑到最简单的one-layer neural network，input是$x$,parameters是$w$和$b$以及一些loss函数。它可以通过Pytorch通过以下方法定义：


In [1]:
import torch

x=torch.ones(5) # input tensor
y=torch.zeros(3) # expected output
w=torch.randn(5,3,requires_grad=True)
b=torch.randn(3,requires_grad=True)
z=torch.matmul(x,w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)


### Tensor,function和Computational graph

上述的代码对应如下的**Computational graph**：
![计算图的结构](https://note.youdao.com/yws/api/personal/file/D65D1CC631F14CF3AD660441D0E3A6F5?method=download&shareKey=db7e6f74684bdc499acfe72db1145849)

在这个neural network中，$w$和$b$是我们需要optimize的parameters，所以我们需要计算带有那些variables的loss function的gradients。为了做到这一点，我们将这些tensor的<kbd>requires_grad</kbd>属性设置为True。 

NOTE

你可以在创建tensor时候使用<kbd>requires_grad</kbd>方法,或者在此之后使用 <kbd>x.requires_grad_(True)</kbd>方法。

我们应用于tensor去构建computational graph的函数实际上是一个function类的Object。这个Object知道如何去在forward function计算这个函数同时也能用在backward propagation的时候计算它的derivative。backward propagation函数的引用存储在tensor的<kbd>grad_fn</kbd> 属性中，你可以参考[该文档](https://pytorch.org/docs/stable/autograd.html#function)以获得更多的信息。

In [2]:
print('Gradient function for z = ',z.grad_fn)
print('Gradient function for loss=',loss.grad_fn)

Gradient function for z =  <AddBackward0 object at 0x000001EBA4C7FC10>
Gradient function for loss= <BinaryCrossEntropyWithLogitsBackward object at 0x000001EBA4C7FCA0>


### 计算Gradients

我们需要计算带parameter的loss function的derivatives去optimize neural network中的这些parameter的weight，也就是说，我们需要在固定的$x$和$y$的值下计算$\frac{\partial loss}{\partial\omega}$和$\frac{\partial loss}{\partial b}$。我们使用<kbd>loss.backward()</kbd>来计算这些derivative。然后再输出<kbd>w.grad</kbd>和<kbd>b.grad</kbd>。


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0509, 0.3229, 0.0148],
        [0.0509, 0.3229, 0.0148],
        [0.0509, 0.3229, 0.0148],
        [0.0509, 0.3229, 0.0148],
        [0.0509, 0.3229, 0.0148]])
tensor([0.0509, 0.3229, 0.0148])


NOTE

我们只能取得computational graph叶节点<kbd>grad</kbd>的属性，这需要将<kbd>required_grad</kbd>属性设置为True。对于graph中的其它的nodes，不能使用gradients。

出于性能的原因，我们只能在给定的graph上使用backward计算gradient calculation一次，如果我们需要在同样的graph上做几次backward，我们需要将<kbd>retain_graph=True</kbd>。

### 禁用Gradient Tracking

默认情况下，所有的tensor属性<kbd>requires_grad=True</kbd>表示他们会追踪他们的computational history和支持gradient computation。然而在一些情况下我们不需要这么做，例如当我们训练一个model只是想用它处理一下input data，或者我们只是想通过network做forward computation。我们可以使用torch.no_grad()围绕在computation code周围来禁用tracking computation。

In [4]:
z=torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
    z=torch.matmul(x,w)+b
print(z.requires_grad)

True
False


使用tensor的<kbd>detach()</kbd>方法也可以达到同样的效果。

In [6]:
z=torch.matmul(x,w)+b
z_det=z.detach()
print(z_det.requires_grad)

False


出现以下几种情况也可以关闭gradient tracking：

* 在neural network的 **frozen parameter**中标记一些parameters。这在微调pretrained network上是一种常用的方法（scenario）。

* 当只做forward pass时为了提高计算速度也可以这么做，因为在tensor的计算上禁用gradient tracking会变得更有效率。

### Computational Graphs
从概念上讲，autograd可以记录data(tensor)和所有的正在进行的操作(以及这些操作后产生的新tensor)到一个包含Function object的有向无环图(directed acyclic graph)中。在这个DAG图中，叶子是input tensor，root是output tensor。通过追踪根到叶，你可以通过chain rule来自动计算gradient。

在前向传播（forward pass），autograd同步做两件事情：

* 运行所需的操作去计算resulting tensor。
* 在DAG图中添加gradient函数的操作

当<kbd>.backward()</kbd>方法在DAG的根上被唤起，开始反向传播（backward pass），autograd接下来会做：

* 计算每个<kbd>.grad_fn</kbd>的gradient
* 将它们加起来保存到各自tensor的<kbd>.grad</kbd>属性中
* 使用chain rule,传播到所有路径直到叶节点

NOTE

**在Pytorch中DAG图是动态的**。需要注意的是当每个<kbd>.backward()</kbd>都执行过后，graph会从头重新创建。autograd开始构建新的graph。这样做就可以让你使用control flow statement早你的model中；如果需要的话，你可以在每一次的迭代中改变shape、sizhe和operation。

### Optional Reading: Tensor Gradients and Jacobian Products

在很多情况下，我们使用的是scalar loss函数，我们需要计算带一些parameter的gradient。然而还有一些情况，output函数是一个arbitrary tensor。在这种情况下，Pytorch允许你计算被称作是Jacobian product，不是实际的gradient。

对于一个向量函数$\vec{y}=f(\vec{x})$,其中$\vec{x}=<x_1,...,x_n>，\vec{y}=<y_1,...,y_n>$,$\vec{y}$关于$\vec{x}$的gradient，用**Jacobian matrix**给出：
$$
J=\left(\begin{matrix}\frac{\partial y_1}{\partial x_1}&\cdots&\frac{\partial y_1}{\partial x_n}\\\vdots&\ddots&\vdots\\\frac{\partial y_m}{\partial x_1}&\cdots&\frac{\partial y_m}{\partial x_n}\\\end{matrix}\right)
$$

取而代之的是计算Jacobian matrix,对于给定的input vector $v=(v_1...v_m)$,Pytorch允许你计算Jacobian product $v^T⋅J$。调用$v$作为augument的<kbd>backward</kbd>来完成这些事情。$v$的size应该和original vector的大小相同，考虑到我们想要计算的product：

In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


注意，当我们使用相同的参数再调用<kbd>backward</kbd>时，gradient的值是不同的。 发生这种情况原因是在backward propagetion时，PyTorch累加了之前的gradient，即计算出来的<kbd>gradient</kbd>的值被添加到computational graph的所有叶节点的<kbd>grad</kbd>属性。 如果要计算正确的gradient，需要先将<kbd>grad</kbd>属性归零。 在实际中，optimizer可以帮助我们做到这一点。

NOTE

以前我们在没有parameters的情况下调用<kbd>backward()</kbd>函数。 这基本上等同于调用backward(torch.tensor(1.0))，这是计算scalar-valued函数的gradient的实用方法，例如训练neural network的loss。

### 拓展阅读
[Autograd Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)